In [1]:
%load_ext autoreload
%autoreload 2
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV, LeaveOneOut
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
#set path to import my_toools.py
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import file_dic, plot_dic
import pandas as pd
import glob


%matplotlib qt

In [2]:
path = '/media/onno/Algemeen//Thesis/GFS_histogram/'

KDE for annual data

In [ ]:

#set path change accordingly
path_file = '/media/onno/Algemeen/Thesis/GFS_histogram/'
path = '/media/onno/Algemeen/Thesis/GFS_KDE/'
variablez = ['envelope','phasespeed','u_wind']
#set latitudes and longitudes for boxes to be used
# latz = [(80,60),(60,40),(40,20)]
# lonz = [(320,340),(320,340),(320,340)]
latz = [(60,40)]
#x axis ranges for KDE to be plotted. These more or less represent physical limits of the corresponding variable
x_rangez = [(0,50),(-10,20),(-20,50)]
#Loop over variable
for k,variable in enumerate(variablez):
    print(variable)
    #Loop over boxes
    for j in range(len(latz)):
        print(latz[j])
        #loop over lead days
        for i in range(11):
            print(i)
            #open file for each box and each lead day
            file = 'GFS_error_weighted_mean_{}_day_{:02d}_lat_lower_left_{}_lon_lower_left_{}.nc'.format(variable,i,latz[j][1],lonz[j][0])
            ds = xr.open_dataset(path_file+file,decode_times=False)
            #convert xarray dataset to numpy array
            values = ds[plot_dic[variable]['data_variable']].values
            #Create KDE between earlier set range. We use a step size of 0.01
            x_range = np.arange(x_rangez[k][0],x_rangez[k][1]+0.01,0.01)
            #Fit KDE to the data. I chose standard bandwith of 1
            #For some reason data must be converted to 2D numpy array
            kde = KernelDensity(bandwidth=1.0, kernel='gaussian')
            kde.fit(values[:,None])
            #get log density data of KDE
            logprob = kde.score_samples(x_range[:,None])
            #save as np array
            np.save(path+file[:-3]+'.npy',logprob)


In [ ]:
#set path change accordingly
path_file = '/home/onno/Thesis/GFS_histogram/seasonal/'
path = '/home/onno/Thesis/GFS_KDE/seasonal/'
#set all variables to be plotted 
variablez = ['envelope','phasespeed','u_wind']
#Loop over each latitude and longitude box
# latz = [(80,60),(60,40),(40,20)]
# lonz = [(320,340),(320,340),(320,340)]
latz = [(60,40)]
lonz = [(290,310)]
#set all seasons
seasonz = ['MAM','JJA','SON','DJF']
#x axis ranges for KDE to be plotted. These more or less represent physical limits of the corresponding variable
x_rangez = [(0,50),(-10,20),(-20,50)]
#Loop over variables
for k,variable in enumerate(variablez):
    print(variable)
    #loop over all boxes
    for j in range(len(latz)):
        print(latz[j])
        #loop over all lead days
        for i in range(11):
            #Because phase speed data at lead day 0 is undefined and not present skip this
            if (variable=='phasespeed')&(i==0):
                continue
            print(i)
            #loop over all seasons
            for season in seasonz:
                #open file
                file = 'GFS_error_weighted_mean_{}_day_{:02d}_lat_lower_left_{}_lon_lower_left_{}_{}.nc'.format(variable,i,latz[j][1],lonz[j][0],season)
                ds = xr.open_dataset(path_file+file,decode_times=False)
                #convert xarray dataset to numpy array
                values = ds[plot_dic[variable]['data_variable']].values
                #Create KDE between earlier set range. We use a step size of 0.01
                x_range = np.arange(x_rangez[k][0],x_rangez[k][1]+0.01,0.01)
                #Fit KDE to the data. I chose standard bandwith of 1
                #For some reason data must be converted to 2D numpy array
                kde = KernelDensity(bandwidth=1.0, kernel='gaussian')
                kde.fit(values[:,None])
                #get log density data of KDE
                logprob = kde.score_samples(x_range[:,None])
                #save as np array
                np.save(path+file[:-3]+'.npy',logprob)


In [5]:
#change font size of presentation
plt.rcParams.update({'font.size': 12})
#set path
path = '/media/onno/Algemeen/Thesis/GFS_KDE/'
#set all variablez
variablez = ['envelope','phasespeed','u_wind']
# set all longitude and latitude boxes
# latz = [(80,60),(60,40),(40,20)]
# lonz = [(320,340),(320,340),(320,340)]
latz = [(60,40)]
lonz = [(290,310)]

dayz = [2,5,8]
y_limz = [3000,3500,2500]
x_rangez = [(0,50),(-10,20),(-20,50)]
for j,variable in enumerate(variablez):
    for i in range(len(latz)):
        fig,ax = plt.subplots(figsize=(16,9))
        file = 'GFS_error_weighted_mean_{}_day_{:02d}_lat_lower_left_{}_lon_lower_left_{}.npy'
        values_day1 = np.load(path+file.format(variable,1,latz[i][1],lonz[i][0]))
        values_day5 = np.load(path+file.format(variable,5,latz[i][1],lonz[i][0]))
        values_day10 = np.load(path+file.format(variable,10,latz[i][1],lonz[i][0]))
        x_range = np.arange(x_rangez[j][0],x_rangez[j][1]+0.01,0.01)
        
#         ds_day2 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_02_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
#                                  decode_times=False)
#         ds_day5 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_05_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
#                                  decode_times=False)
#         ds_day8 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_08_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
#                                  decode_times=False)
#         ax.hist(x=[ds_day2[plot_dic[variable]['data_variable']].values,ds_day5[plot_dic[variable]['data_variable']].values,ds_day8[plot_dic[variable]['data_variable']].values],
#                 bins=20,range=rangez[j],color=['purple','red','orange'],density=True)
        ax.fill_between(x_range,np.exp(values_day1),alpha=.5,color='purple')
#         ax.fill_between(x_range,np.exp(values_day5),alpha=.5,color='red')
        ax.fill_between(x_range,np.exp(values_day10),alpha=.5,color='orange')

        ax.legend(['Day 1','Day 10'])
#         ax.set_ylim([0,y_limz[j]])
#         ax.set_ylabel('N')
        ax.set_xlabel(plot_dic[variable]['label'])
        fig.suptitle('GEFS KDE Distribution of {} at {}-{} N° and {}-{} E°'.format(plot_dic[variable]['title'],
                                                                      latz[i][1],latz[i][0],
                                                                      lonz[i][0],lonz[i][1]))
        sys.exit()
        fig.savefig(path+'/KDE_{}_lat_lower_left_{}_lon_lower_left_{}_norm.png'.format(variable,latz[i][1],lonz[i][0]))
        plt.close(fig)

SystemExit: 

/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


KDE for seasonal data

In [57]:
plt.rcParams.update({'font.size': 12})
path = '/home/onno/Thesis/GFS_KDE/seasonal/'
variablez = ['envelope','phasespeed','u_wind']
latz = [(80,60),(60,40),(40,20)]
lonz = [(320,340),(320,340),(320,340)]
dayz = [2,5,8]
seasonz = ['MAM','JJA','SON','DJF']
y_limz = [3000,3500,2500]
x_rangez = [(0,50),(-10,20),(-20,50)]
for j,variable in enumerate(variablez):
    if variable == 'phasespeed':
        continue
    for i in range(len(latz)):
        for season in seasonz:
            fig,ax = plt.subplots(figsize=(16,9))
            file = 'GFS_error_weighted_mean_{}_day_{:02d}_lat_lower_left_{}_lon_lower_left_{}_{}.npy'
            values_day2 = np.load(path+file.format(variable,2,latz[i][1],lonz[i][0],season))
            values_day5 = np.load(path+file.format(variable,5,latz[i][1],lonz[i][0],season))
            values_day8 = np.load(path+file.format(variable,8,latz[i][1],lonz[i][0],season))
            x_range = np.arange(x_rangez[j][0],x_rangez[j][1]+0.01,0.01)

    #         ds_day2 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_02_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
    #                                  decode_times=False)
    #         ds_day5 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_05_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
    #                                  decode_times=False)
    #         ds_day8 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_08_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
    #                                  decode_times=False)
    #         ax.hist(x=[ds_day2[plot_dic[variable]['data_variable']].values,ds_day5[plot_dic[variable]['data_variable']].values,ds_day8[plot_dic[variable]['data_variable']].values],
    #                 bins=20,range=rangez[j],color=['purple','red','orange'],density=True)
            ax.fill_between(x_range,np.exp(values_day2),alpha=.5,color='purple')
    #         ax.fill_between(x_range,np.exp(values_day5),alpha=.5,color='red')
            ax.fill_between(x_range,np.exp(values_day8),alpha=.5,color='orange')

            ax.legend(['Day 1','Day 10'])
    #         ax.set_ylim([0,y_limz[j]])
    #         ax.set_ylabel('N')
            ax.set_xlabel(plot_dic[variable]['label'])
            fig.suptitle('GEFS KDE Distribution of {} at {}-{} N° and {}-{} E° {}'.format(plot_dic[variable]['title'],
                                                                          latz[i][1],latz[i][0],
                                                                          lonz[i][0],lonz[i][1],season))
            fig.savefig(path+'/KDE_{}_lat_lower_left_{}_lon_lower_left_{}_{}.png'.format(variable,latz[i][1],lonz[i][0],season))
            plt.close(fig)

In [32]:
plt.rcParams.update({'font.size': 12})
path = '/home/onno/Thesis/GFS_KDE/seasonal/'
variablez = ['envelope','phasespeed','u_wind']
latz = [(80,60),(60,40),(40,20)]
lonz = [(320,340),(290,310),(320,340)]
dayz = [2,5,8]
seasonz = ['MAM','JJA','SON','DJF']
y_limz = [0.12,0.25,0.08]
y_limz_JJA = [0.16,0.3,0.12]
x_rangez = [(0,50),(-10,20),(-20,50)]
for j,variable in enumerate(variablez):
    for season in seasonz:
        fig,axz=plt.subplots(1,3,figsize=(16,9),sharey=True)
        for i in range(len(latz)):
            ax = axz.flat[i]
            file = 'GFS_error_weighted_mean_{}_day_{:02d}_lat_lower_left_{}_lon_lower_left_{}_{}.npy'
            values_day1 = np.load(path+file.format(variable,1,latz[i][1],lonz[i][0],season))
            values_day5 = np.load(path+file.format(variable,5,latz[i][1],lonz[i][0],season))
            values_day10 = np.load(path+file.format(variable,10,latz[i][1],lonz[i][0],season))
            x_range = np.arange(x_rangez[j][0],x_rangez[j][1]+0.01,0.01)

    #         ds_day2 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_02_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
    #                                  decode_times=False)
    #         ds_day5 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_05_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
    #                                  decode_times=False)
    #         ds_day8 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_08_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
    #                                  decode_times=False)
    #         ax.hist(x=[ds_day2[plot_dic[variable]['data_variable']].values,ds_day5[plot_dic[variable]['data_variable']].values,ds_day8[plot_dic[variable]['data_variable']].values],
    #                 bins=20,range=rangez[j],color=['purple','red','orange'],density=True)
            ax.fill_between(x_range,np.exp(values_day1),alpha=.5,color='purple')
    #         ax.fill_between(x_range,np.exp(values_day5),alpha=.5,color='red')
            ax.fill_between(x_range,np.exp(values_day10),alpha=.5,color='orange')

            if i == len(latz)-1:
                ax.legend(['Day 1','Day 10'])
        #         ax.set_ylim([0,y_limz[j]])
        #         ax.set_ylabel('N')
            ax.set_xlabel(plot_dic[variable]['label'])
            ax.set_title('{}-{} °N {}-{} °E'.format(latz[i][1],latz[i][0],
                                lonz[i][0],lonz[i][1]))
            if i == 0:
                axz[i].yaxis.set_ticks_position('left')
            else:
                axz[i].yaxis.set_ticks_position('none')

        fig.subplots_adjust(left=0.05,right=0.99,wspace=0)
        fig.suptitle('GEFS KDE Distribution of {} in {}'.format(plot_dic[variable]['title'],season))
        fig.savefig(path+'/KDE_{}_{}.png'.format(variable,season))
        plt.close(fig)

In [8]:
plt.rcParams.update({'font.size': 12})
path = '/media/onno/Algemeen/Thesis/GFS_KDE/seasonal/'
variablez = ['envelope','phasespeed','u_wind']
latz = [(80,60),(60,40),(40,20)]
lonz = [(320,340),(290,310),(320,340)]
dayz = [2,5,8]
# seasonz = ['MAM','JJA','SON','DJF']
seasonz = ['JJA']
y_limz = [0.12,0.25,0.08]
y_limz_JJA = [0.16,0.3,0.12]
y_limz_DJF = [0.1,0.2,0.06]
x_rangez = [(0,50),(-10,20),(-20,50)]
for j,variable in enumerate(variablez):
    for season in seasonz:
        fig,axz=plt.subplots(3,1,figsize=(16,9),sharex=True)
        for i in range(len(latz)):
            ax = axz.flat[i]
            file = 'GFS_error_weighted_mean_{}_day_{:02d}_lat_lower_left_{}_lon_lower_left_{}_{}.npy'
            values_day1 = np.load(path+file.format(variable,1,latz[i][1],lonz[i][0],season))
            values_day5 = np.load(path+file.format(variable,5,latz[i][1],lonz[i][0],season))
            values_day10 = np.load(path+file.format(variable,10,latz[i][1],lonz[i][0],season))
            x_range = np.arange(x_rangez[j][0],x_rangez[j][1]+0.01,0.01)

    #         ds_day2 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_02_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
    #                                  decode_times=False)
    #         ds_day5 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_05_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
    #                                  decode_times=False)
    #         ds_day8 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_08_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
    #                                  decode_times=False)
    #         ax.hist(x=[ds_day2[plot_dic[variable]['data_variable']].values,ds_day5[plot_dic[variable]['data_variable']].values,ds_day8[plot_dic[variable]['data_variable']].values],
    #                 bins=20,range=rangez[j],color=['purple','red','orange'],density=True)
            ax.fill_between(x_range,np.exp(values_day1),alpha=.5,color='purple')
    #         ax.fill_between(x_range,np.exp(values_day5),alpha=.5,color='red')
            ax.fill_between(x_range,np.exp(values_day10),alpha=.5,color='orange')

            if i == 0:
                ax.legend(['Day 1','Day 10'])
        #         ax.set_ylim([0,y_limz[j]])
        #         ax.set_ylabel('N')
            if i == len(latz)-1:
                ax.set_xlabel(plot_dic[variable]['label'])
            ax.set_title('{}-{} °N {}-{} °E'.format(latz[i][1],latz[i][0],
                                lonz[i][0],lonz[i][1]))
            if i == len(latz)-1:
                axz[i].xaxis.set_ticks_position('bottom')
            else:
                axz[i].xaxis.set_ticks_position('none')
            if season == 'JJA':
                ax.set_ylim([0,y_limz_JJA[j]])
            elif season == 'DJF':
                ax.set_ylim([0,y_limz_DJF[j]])
            else:
                ax.set_ylim([0,y_limz[j]])

        fig.subplots_adjust(left=0.05,right=0.99,wspace=0)
        fig.suptitle('GEFS Kernel Density Estimation of {} in {}'.format(plot_dic[variable]['title'],season))
        fig.savefig(path+'sharex/KDE_{}_{}_sharex.png'.format(variable,season),dpi=300)
        plt.close(fig)

In [33]:
plt.rcParams.update({'font.size': 12})
path = '/home/onno//Thesis/GFS_KDE/'
variablez = ['envelope','phasespeed','u_wind']
latz = [(80,60),(60,40),(40,20)]
lonz = [(320,340),(290,310),(320,340)]
dayz = [2,5,8]
y_limz = [3000,3500,2500]
x_rangez = [(0,50),(-10,20),(-20,50)]
for j,variable in enumerate(variablez):

    fig,axz=plt.subplots(1,3,figsize=(16,9),sharey=True)
    for i in range(len(latz)):
        ax = axz.flat[i]
        file = 'GFS_error_weighted_mean_{}_day_{:02d}_lat_lower_left_{}_lon_lower_left_{}.npy'
        values_day1 = np.load(path+file.format(variable,1,latz[i][1],lonz[i][0]))
        values_day5 = np.load(path+file.format(variable,5,latz[i][1],lonz[i][0]))
        values_day10 = np.load(path+file.format(variable,10,latz[i][1],lonz[i][0]))
        x_range = np.arange(x_rangez[j][0],x_rangez[j][1]+0.01,0.01)
        
#         ds_day2 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_02_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
#                                  decode_times=False)
#         ds_day5 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_05_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
#                                  decode_times=False)
#         ds_day8 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_08_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
#                                  decode_times=False)
#         ax.hist(x=[ds_day2[plot_dic[variable]['data_variable']].values,ds_day5[plot_dic[variable]['data_variable']].values,ds_day8[plot_dic[variable]['data_variable']].values],
#                 bins=20,range=rangez[j],color=['purple','red','orange'],density=True)
        ax.fill_between(x_range,np.exp(values_day1),alpha=.5,color='purple')
#         ax.fill_between(x_range,np.exp(values_day5),alpha=.5,color='red')
        ax.fill_between(x_range,np.exp(values_day10),alpha=.5,color='orange')

        if i == len(latz)-1:
            ax.legend(['Day 1','Day 10'])
    #         ax.set_ylim([0,y_limz[j]])
    #         ax.set_ylabel('N')
        ax.set_xlabel(plot_dic[variable]['label'])
        ax.set_title('{}-{} °N {}-{} °E'.format(latz[i][1],latz[i][0],
                            lonz[i][0],lonz[i][1]))
        if i == 0:
            axz[i].yaxis.set_ticks_position('left')
        else:
            axz[i].yaxis.set_ticks_position('none')

    fig.subplots_adjust(left=0.05,right=0.99,wspace=0)
    fig.suptitle('GEFS KDE Distribution of {}'.format(plot_dic[variable]['title']))
    fig.savefig(path+'/KDE_{}.png'.format(variable))
    plt.close(fig)

In [18]:
plt.rcParams.update({'font.size': 12})
path = '/home/onno//Thesis/GFS_KDE/'
variablez = ['envelope','phasespeed','u_wind']
latz = [(80,60),(60,40),(40,20)]
lonz = [(320,340),(320,340),(320,340)]
dayz = [2,5,8]
y_limz = [0.1,0.2,0.06]
x_rangez = [(0,50),(-10,20),(-20,50)]
for j,variable in enumerate(variablez):

    fig,axz=plt.subplots(3,1,figsize=(16,9),sharex=True)
    for i in range(len(latz)):
        ax = axz.flat[i]
        file = 'GFS_error_weighted_mean_{}_day_{:02d}_lat_lower_left_{}_lon_lower_left_{}.npy'
        values_day1 = np.load(path+file.format(variable,1,latz[i][1],lonz[i][0]))
        values_day5 = np.load(path+file.format(variable,5,latz[i][1],lonz[i][0]))
        values_day10 = np.load(path+file.format(variable,10,latz[i][1],lonz[i][0]))
        x_range = np.arange(x_rangez[j][0],x_rangez[j][1]+0.01,0.01)
        
#         ds_day2 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_02_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
#                                  decode_times=False)
#         ds_day5 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_05_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
#                                  decode_times=False)
#         ds_day8 = xr.open_dataset(path+'/GFS_error_weighted_mean_{}_day_08_lat_lower_left_{}_lon_lower_left_{}_attempt2.nc'.format(variable,latz[i][1],lonz[i][0]),
#                                  decode_times=False)
#         ax.hist(x=[ds_day2[plot_dic[variable]['data_variable']].values,ds_day5[plot_dic[variable]['data_variable']].values,ds_day8[plot_dic[variable]['data_variable']].values],
#                 bins=20,range=rangez[j],color=['purple','red','orange'],density=True)
        ax.fill_between(x_range,np.exp(values_day1),alpha=.5,color='purple')
#         ax.fill_between(x_range,np.exp(values_day5),alpha=.5,color='red')
        ax.fill_between(x_range,np.exp(values_day10),alpha=.5,color='orange')

        if i == 0:
            ax.legend(['Day 1','Day 10'])
    #         ax.set_ylim([0,y_limz[j]])
    #         ax.set_ylabel('N')
        if i == len(latz)-1:
            ax.set_xlabel(plot_dic[variable]['label'])
        ax.set_title('{}-{} °N {}-{} °E'.format(latz[i][1],latz[i][0],
                            lonz[i][0],lonz[i][1]))
        if i == len(latz)-1:
            axz[i].xaxis.set_ticks_position('bottom')
        else:
            axz[i].xaxis.set_ticks_position('none')
        ax.set_ylim([0,y_limz[j]])

    fig.subplots_adjust(left=0.05,right=0.99,wspace=0)
    fig.suptitle('GEFS KDE Distribution of {}'.format(plot_dic[variable]['title']))
    fig.savefig(path+'/KDE_{}_share_x.png'.format(variable))
    plt.close(fig)